In [1]:
import requests
import pandas as pd
import time

In [2]:
#### Create list for product information
### in generals
product_name = []   
manufacturer = []
user_purpose = []
hdd_ssd = []
laptop_cpu = []
screen_size = []
laptop_vga = []
condition = []
### In region
original_price = []
promotion_price = []
### In filterable 
laptop_ram = []
resolution = []

In [3]:
### There is an element named "page" in json_data which is identfied for number of "show more items" times.
### Using f-string to loop the "i" for "page"'s value

for i in range(1,77):
    headers = {
        'authority': 'api.cellphones.com.vn',
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9',
        'origin': 'https://cellphones.com.vn',
        'referer': 'https://cellphones.com.vn/',
        'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
        }
    json_data = {
                'query': '\n            query{\n                products(\n                        filter: {\n                            static: {\n                                categories: ["380"],\n                                region_id: 7, \n                                stock: {\n                                    from: 0\n                                },\n                               filter_price: {from:0to:179990000}\n                            },\n                            dynamic: {\n                                \n                            }\n                        },\n                        page: no,\n                        size: 20,\n                        sort: [{view: desc}]\n                    )\n                {\n                    general{\n                        product_id\n                        name\n                        attributes\n                    },\n                    filterable{\n                        is_installment\n                        stock_available\n                        filter {\n                           id\n                           Label\n                        }\n                    },\n                    region{\n                        price\n                        special_price\n                        promotion_information\n                        thumbnail\n                        promotion_pack\n                    }\n                }\n            }',
                'variables': {},
            }

    jdq = json_data['query'].replace('no',str(i))
    new_json_data = {'query' : jdq,'variables': {},}
    response = requests.post('https://api.cellphones.com.vn/graphql/query', headers=headers, json=new_json_data)
    results_json = response.json()
    list_of_products_json = results_json['data']['products']
    
    ## Convert list to dict for easier looking up
    def list_to_dict(lis):
        dic = {}
        for i in lis:
            dic_val = {list(i.values())[0] : list(i.values())[1]}
            dic.update(dic_val)
        return dic
    
    for product_result in list_of_products_json:
        ### Get ram and resolution values from ['filter_list'] ##
        dic = list_to_dict(product_result['filterable']['filter'])
        if 'laptop_ram' in dic.keys():
            laptop_ram.append(dic['laptop_ram'])
        elif 'laptop_ram' not in dic.keys():
            laptop_ram.append('None')

        if 'laptop_resolution_filter' in dic.keys():
            resolution.append(dic['laptop_resolution_filter'])
        elif 'laptop_resolution_filter' not in dic.keys():
            resolution.append('None')

        ### Get values from ['region'] ###
        
        ## original_price
        try:
            original_price.append(int(product_result['region']['price']))
        except:
            original_price.append('None')
            
        ## promotion_price
        try:
            promotion_price.append(int(product_result['region']['special_price']))
        except:
            promotion_price.append('None')
            
        ### Get values from ['general'] ###
        
        ##name
        try:
            product_name.append(product_result['general']['name'])
        except:
            product_name.append('None')
            
        ##manufacturer
        try:
            manufacturer.append(product_result['general']['attributes']['manufacturer'])
        except:
            manufacturer.append('None')
            
        ##user purpose
        try:
            user_purpose.append(product_result['general']['attributes']['nhu_cau_su_dung'])
        except:
            user_purpose.append('None')
            
        ## hdd_ssd
        try:
            hdd_ssd.append(product_result['general']['attributes']['hdd_sdd'])
        except:
            hdd_ssd.append('None')
            
        ## laptop_cpu
        try:
            laptop_cpu.append(product_result['general']['attributes']['laptop_cpu'])
        except:
            laptop_cpu.append('None')
            
        ## screen_size
        try:
            screen_size.append(product_result['general']['attributes']['laptop_screen_size_filter'])
        except:
            screen_size.append('None')
            
        ## laptop_vga
        try:
            laptop_vga.append(product_result['general']['attributes']['laptop_vga_filter'])
        except:
            laptop_vga.append('None')
            
        ## condition
        try:
            condition.append(product_result['general']['attributes']['product_condition'])
        except:
            condition.append('None')

    time.sleep(10)                  

In [4]:
df_products = pd.DataFrame({'Name': product_name, 'Manufacturer': manufacturer, 'Laptop_CPU': laptop_cpu, 
                            'RAM': laptop_ram, 'HDD_SSD': hdd_ssd, 'VGA': laptop_vga,
                            'Screen_size': screen_size, 'Resolution': resolution ,'User_purpose' : user_purpose,
                             'Condition': condition, 'Original_price': original_price,'Promotion_price': promotion_price})

In [5]:
df_products

,Name,Manufacturer,Laptop_CPU,RAM,HDD_SSD,VGA,Screen_size,Resolution,User_purpose,Condition,Original_price,Promotion_price
0,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,Apple,Apple M1,8GB,SSD: 256GB,Card Onboard,Khoảng 13 inch,Retina,"Cao cấp - Sang trọng, Mỏng nhẹ",Mới,28990000,22290000
1,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chín...,Apple,Apple M2,8GB,SSD: 256GB,AMD Radeon Series,Khoảng 13 inch,None,"Học tập - Văn phòng, Đồ họa - Kỹ thuật, Cao cấ...",Mới,35990000,32990000
2,Apple MacBook Pro 13 Touch Bar M1 256GB 2020 I...,Apple,Apple M1,8GB,SSD: 256GB,Card Onboard,Khoảng 13 inch,Retina,Đồ họa - Kỹ thuật,Mới,35990000,28990000
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,Apple,Apple M2,8GB,SSD: 256GB,Card Onboard,Khoảng 13 inch,None,"Học tập - Văn phòng, Cao cấp - Sang trọng",Mới,32990000,31490000
4,Laptop Asus Gaming Rog Strix G15 G513IH HN015W,ASUS,AMD Ryzen 7,8GB,SSD: 512GB,NVIDIA GeForce Series,Trên 15 inch,Full HD,"Đồ họa - Kỹ thuật, Gaming",Mới,23990000,17990000
5,Laptop Asus Vivobook 13 Slate Oled T3300KA,ASUS,Intel Celeron / Pentinum,8GB,SSD: 256GB,Card Onboard,Khoảng 13 inch,None,"Học tập - Văn phòng, Mỏng nhẹ",Mới,17790000,14790000
6,Laptop Gaming MSI Bravo 15 B5DD 276VN,MSI,AMD Ryzen 5,8GB,SSD: 512GB,AMD Radeon Series,Trên 15 inch,Full HD,"Đồ họa - Kỹ thuật, Gaming",Mới,22490000,14890000
7,Laptop MSI Modern 14 B11MOU 1030VN,MSI,Intel Core i3,8GB,SSD: 256GB,Card Onboard,Khoảng 14 inch,Full HD,"Học tập - Văn phòng, Mỏng nhẹ",Mới,14990000,11790000
8,Laptop Gaming Acer Nitro 5 AN515 45 R6EV,Acer,AMD Ryzen 5,8GB,SSD: 512GB,NVIDIA GeForce Series,Trên 15 inch,Full HD,"Đồ họa - Kỹ thuật, Gaming",Mới,23990000,18990000
9,Laptop HP 14-CF2033WM 3V7G4UA,HP,Intel Celeron / Pentinum,4GB,SSD: 128GB,Card Onboard,Khoảng 14 inch,HD,Học tập - Văn phòng,Mới,11990000,6490000


In [14]:
df_products[df_products["Laptop_CPU"] == 'None']

,Name,Manufacturer,Laptop_CPU,RAM,HDD_SSD,VGA,Screen_size,Resolution,User_purpose,Condition,Original_price,Promotion_price
45,Laptop Asus Rog Strix G15 G513IM HN008W,ASUS,None,16GB,None,None,None,None,"Đồ họa - Kỹ thuật, Gaming",Mới,34990000,27990000
125,Laptop ASUS Gaming ROG Zephyrus G14 GA401I,ASUS,None,None,None,None,None,None,Gaming,Mới,26990000,26990000
173,Surface Laptop 3 Core i5 / 8GB / 128 GB / 13.5...,Microsoft,None,None,SSD: 512GB,None,None,None,Cao cấp - Sang trọng,Mới,26990000,22490000
180,Laptop Asus Tuf Gaming F15 FX507ZM HN123W,ASUS,None,None,None,None,None,None,None,Mới,39990000,34990000
186,Apple Studio Display 27 5K Chân đế cố định 202...,Apple,None,None,None,None,None,None,None,Mới,43990000,0
315,Laptop ASUS TUF Gaming FX505DT-HN478T,ASUS,None,None,None,None,None,None,Gaming,Mới,19290000,17790000
321,Laptop Acer Nitro 5 AN515-55-5923 (NH.Q7NSV.004),Acer,None,8GB,None,None,None,None,Gaming,Mới,23990000,19590000
348,Laptop Lenovo Yoga Slim 7 14ITL05,Lenovo,None,None,None,None,None,None,None,Mới,23990000,20490000
364,Laptop Acer Gaming Aspire 7 A715-41G-R150,Acer,None,8GB,None,None,None,None,Gaming,Mới,21490000,19160000
369,Laptop HP Evy 13 BA1047 290F5UA Nhập Khẩu chín...,HP,None,8GB,SSD: 256GB,Card Onboard,None,None,"Học tập - Văn phòng, Mỏng nhẹ",Mới,22990000,17490000


In [29]:
df_products.loc[45:85]

,Name,Manufacturer,Laptop_CPU,RAM,HDD_SSD,VGA,Screen_size,Resolution,User_purpose,Condition,Original_price,Promotion_price
45,Laptop Asus Rog Strix G15 G513IM HN008W,ASUS,None,16GB,None,None,None,None,"Đồ họa - Kỹ thuật, Gaming",Mới,34990000,27990000
46,Laptop Fujitsu UH X 9U13A2 4ZR1G97610,Hãng khác,Intel Core i7,16GB,SSD: 1TB,Card Onboard,Khoảng 13 inch,Full HD,"Học tập - Văn phòng, Mỏng nhẹ",Mới,35990000,32990000
47,Laptop Lenovo Ideapad 3,Lenovo,Intel Core i3,4GB,SSD: 128GB,Card Onboard,None,Full HD,Học tập - Văn phòng,Mới,16990000,7990000
48,Surface Pro 8 Core i5 / 8GB / 128GB Nhập khẩu ...,Microsoft,Intel Core i5,8GB,SSD: 128GB,None,Khoảng 13 inch,None,"Học tập - Văn phòng, Cao cấp - Sang trọng, Lap...",Mới,35990000,24990000
49,Apple MacBook Pro 13 M2 2022 16GB 512GB I Chín...,Apple,Apple M2,16GB,SSD: 512GB,AMD Radeon Series,Khoảng 13 inch,None,"Học tập - Văn phòng, Đồ họa - Kỹ thuật, Cao cấ...",Mới,46990000,44990000
50,Laptop Acer Travelmate TMB311-31-P49D N20H1_NX...,Acer,Intel Celeron / Pentinum,4GB,SSD: 256GB,Card Onboard,None,HD,Học tập - Văn phòng,Mới,11490000,4690000
51,Laptop Asus Gaming TUF FX506LH HN188W,ASUS,Intel Core i5,8GB,SSD: 512GB,NVIDIA GeForce Series,Trên 15 inch,Full HD,"Đồ họa - Kỹ thuật, Gaming",Mới,21990000,15690000
52,Laptop MSI Modern 14 B5M 204VN,MSI,AMD Ryzen 5,8GB,SSD: 512GB,None,Khoảng 14 inch,Full HD,"Học tập - Văn phòng, Mỏng nhẹ",Mới,17990000,13190000
53,Laptop Lenovo Ideapad 5 PRO 14ACN6 82L700MAVN,Lenovo,AMD Ryzen 7,16GB,SSD: 512GB,Card Onboard,Khoảng 14 inch,2K (Quad HD),"Cao cấp - Sang trọng, Mỏng nhẹ",Mới,23990000,19990000
54,Laptop Asus TUF Gaming F15 FX506HC-HN002T,ASUS,Intel Core i5,8GB,SSD: 512GB,NVIDIA GeForce Series,Trên 15 inch,Full HD,"Đồ họa - Kỹ thuật, Gaming",Mới,25990000,24390000


In [30]:
df_products.to_csv('laptop_cellphones_data.csv', sep=',', index = True)

In [32]:
df_products.to_excel('laptop_cellphones_data_excel.xlsx', index = True)